Segmenting and Clustering Neighborhoods in Toronto:

Before we get the data and start exploring it, let's download all the dependencies that we will need

In [6]:
import numpy as np #vectorized manner

import pandas as pd #data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #JSON files


from geopy.geocoders import Nominatim #latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup


print('Libraries imported.')

Libraries imported.


In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

Getting page data from Wikipedia

In [7]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

#beautifulsoup to parse the HTML/XML code
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

Extracting table data from Wikipedia

In [8]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

#print(table)

#extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


Postal codes with more than 1 neighborhoods

In [9]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


Pandas Dataframe

In [10]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M6M,York,"Keelesdale, Mount Dennis, Silverthorn"
1,M1G,Scarborough,Woburn
2,M4E,East Toronto,The Beaches
3,M4C,East York,Woodbine Heights
4,M6H,West Toronto,Dovercourt Village
5,M1B,Scarborough,"Rouge, Malvern"
6,M2K,North York,Bayview Village
7,M1M,Scarborough,"Cliffcrest, Cliffside"
8,M2M,North York,"Newtonbrook, Willowdale"
9,M3H,North York,"Bathurst Manor, Wilson Heights"


In [11]:
df_toronto.shape

(77, 3)

Read the Geospatial csv file and inner join it

In [12]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['Postcode', 'Latitude', 'Longitude']

In [13]:
df_join = pd.merge(df_toronto, df_latlng, on=['Postcode'], how='inner')

In [14]:
df_join.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6M,York,"Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
1,M1G,Scarborough,Woburn,43.770992,-79.216917
2,M4E,East Toronto,The Beaches,43.676357,-79.293031
3,M4C,East York,Woodbine Heights,43.695344,-79.318389
4,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259


Further Analysis:

In [18]:
#1 Unique values
print('The dataframe has {} boroughs and {} neighborhood.'.format(
        len(df_join['Borough'].unique()),
        df_join.shape[0]))

The dataframe has 9 boroughs and 77 neighborhood.


In [19]:
#2 Toronto coordinates
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
#3 Toronto map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [26]:
for lat, lng, borough, neighborhood in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto